In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:

# Read the CSV file
training_data = pd.read_csv("data/training_dataPhil.csv", index_col=0)

training_data.head()

,sentence,difficulty,lexical_complexite,note_orthographe,char_length,word_length,type_token_ratio,sentence_length,avg_word_length,complexite_texte,...,DET,PRON,NUM,NOUN,INTJ,ADP,ADJ,VERB,PROPN,SCONJ
id,,,,,,,,,,,,,,,,,,,,,
0,Les coûts kilométriques réels peuvent diverger...,4,0.194007,1.000000,0.160077,0.140152,0.467105,0.140152,0.339713,0.244565,...,0.066667,0.000000,0.000000,0.311111,0.0,0.288889,0.066667,0.088889,0.0000,0.0
1,"Le bleu, c'est ma couleur préférée mais je n'a...",0,0.082334,1.000000,0.036990,0.041667,1.000000,0.041667,0.204545,0.086957,...,0.187500,0.125000,0.000000,0.125000,0.0,0.000000,0.000000,0.125000,0.0625,0.0
2,Le test de niveau en français est sur le site ...,0,0.088078,0.769231,0.039541,0.045455,0.826923,0.045455,0.195804,0.081522,...,0.200000,0.000000,0.000000,0.400000,0.0,0.266667,0.000000,0.066667,0.0000,0.0
3,Est-ce que ton mari est aussi de Boston?,0,0.062664,1.000000,0.022959,0.026515,1.000000,0.026515,0.193182,0.054348,...,0.000000,0.100000,0.000000,0.300000,0.0,0.100000,0.000000,0.100000,0.1000,0.1
4,"Dans les écoles de commerce, dans les couloirs...",2,0.184993,1.000000,0.130740,0.125000,0.602941,0.125000,0.288770,0.228261,...,0.095238,0.047619,0.047619,0.238095,0.0,0.261905,0.047619,0.095238,0.0000,0.0


Modèle pré-entraîné RoBERTa en français

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# Utilisez un modèle pré-entraîné en français
model_name = "jplu/tf-camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, from_tf=True)  # Spécifiez from_tf=True ici

phrases = training_data['sentence'].tolist()

# Utilisez le tokenizer pour convertir les phrases en tokens BERT
tokenized_phrases = tokenizer(phrases, padding=True, truncation=True, return_tensors="pt", max_length=128)

# Passez les tokens encodés au modèle BERT pour obtenir les embeddings
with torch.no_grad():
    embeddings = model(**tokenized_phrases)

bert_embeddings = embeddings.last_hidden_state


All TF 2.0 model weights were used when initializing CamembertModel.

All the weights of CamembertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use CamembertModel for predictions without further training.


AVEC PCA

In [4]:
from sklearn.decomposition import PCA

# Supposons que bert_embeddings contient les embeddings BERT sous forme de tableau tridimensionnel
# (nombre d'exemples x longueur de phrase x dimension de l'embedding)

# Aplatissez les embeddings pour les rendre bidimensionnels
num_examples, seq_length, embedding_dim = bert_embeddings.shape
bert_embeddings_2d = bert_embeddings.reshape(num_examples, -1)  # -1 signifie que numpy doit calculer automatiquement la dimension

# Appliquez PCA aux embeddings bidimensionnels
num_components = 50  # Choisissez le nombre de composantes principales souhaité
pca = PCA(n_components=num_components)
reduced_embeddings = pca.fit_transform(bert_embeddings_2d)
del pca

Avec t-SNE

In [12]:
from sklearn.manifold import TSNE

# Supposons que bert_embeddings contient les embeddings BERT sous forme de tableau tridimensionnel
# (nombre d'exemples x longueur de phrase x dimension de l'embedding)

# Aplatissez les embeddings pour les rendre bidimensionnels
num_examples, seq_length, embedding_dim = bert_embeddings.shape
bert_embeddings_2d = bert_embeddings.reshape(num_examples, -1)  # -1 signifie que numpy doit calculer automatiquement la dimension

# Appliquez t-SNE aux embeddings bidimensionnels sans spécifier n_components
tsne = TSNE()
reduced_embeddings = tsne.fit_transform(bert_embeddings_2d)
del tsne

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Avec UMAP

In [20]:
import umap.umap_ as umap

# Supposons que bert_embeddings contient les embeddings BERT sous forme de tableau tridimensionnel
# (nombre d'exemples x longueur de phrase x dimension de l'embedding)

# Aplatissez les embeddings pour les rendre bidimensionnels
num_examples, seq_length, embedding_dim = bert_embeddings.shape
bert_embeddings_2d = bert_embeddings.reshape(num_examples, -1)  # -1 signifie que numpy doit calculer automatiquement la dimension

# Appliquez UMAP aux embeddings bidimensionnels avec le nombre de composantes souhaité
num_components = 2  # Choisissez le nombre de composantes principales souhaité (2 pour réduire en 2D)
reducer = umap.UMAP(n_components=num_components)
reduced_embeddings = reducer.fit_transform(bert_embeddings_2d)

del reducer


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Divison des données

In [23]:
from sklearn.model_selection import train_test_split
import numpy as np

training_data_x = training_data.drop(columns=["difficulty", "sentence"])
X_combined = np.concatenate((reduced_embeddings, training_data_x), axis=1)

X = X_combined  # Les embeddings réduits après PCA
y = training_data["difficulty"]  # Les niveaux de difficulté (étiquettes)

# Effectuez la division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Régression Logistique :

In [24]:
from sklearn.linear_model import LogisticRegression

# Créez un modèle de régression logistique
logistic_regression_model = LogisticRegression()

# Entraînez le modèle sur l'ensemble d'entraînement
logistic_regression_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = logistic_regression_model.score(X_test, y_test)
print("Précision du modèle de régression logistique :", accuracy)

del logistic_regression_model

Précision du modèle de régression logistique : 0.4166666666666667


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM (Support Vector Machine) :

In [25]:
from sklearn.svm import SVC

# Créez un modèle SVM (classification)
svm_model = SVC()

# Entraînez le modèle sur l'ensemble d'entraînement
svm_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = svm_model.score(X_test, y_test)
print("Précision du modèle SVM :", accuracy)

del svm_model

Précision du modèle SVM : 0.3625


Arbres de Décision :

In [26]:
from sklearn.tree import DecisionTreeClassifier

# Créez un modèle d'arbre de décision
decision_tree_model = DecisionTreeClassifier()

# Entraînez le modèle sur l'ensemble d'entraînement
decision_tree_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = decision_tree_model.score(X_test, y_test)
print("Précision du modèle d'arbre de décision :", accuracy)

del decision_tree_model

Précision du modèle d'arbre de décision : 0.34270833333333334


Forêts Aléatoires :

In [27]:
from sklearn.ensemble import RandomForestClassifier

# Créez un modèle de forêt aléatoire
random_forest_model = RandomForestClassifier()

# Entraînez le modèle sur l'ensemble d'entraînement
random_forest_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = random_forest_model.score(X_test, y_test)
print("Précision du modèle de forêt aléatoire :", accuracy)

del random_forest_model

Précision du modèle de forêt aléatoire : 0.4322916666666667


Réseaux de Neurones (utilisant scikit-learn) :

In [28]:
from sklearn.neural_network import MLPClassifier

# Créez un modèle de réseau de neurones (multilayer perceptron)
mlp_model = MLPClassifier()

# Entraînez le modèle sur l'ensemble d'entraînement
mlp_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = mlp_model.score(X_test, y_test)
print("Précision du modèle de réseau de neurones :", accuracy)

del mlp_model

Précision du modèle de réseau de neurones : 0.42916666666666664


/Users/phil/anaconda3/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Réseaux de Neurones Récursifs - RNN

In [29]:
# Créez votre modèle de classification (par exemple, Réseaux de Neurones Récursifs - RNN)
from sklearn.neural_network import MLPClassifier
rnn_model = MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=1000, random_state=42)

# Entraînez le modèle sur l'ensemble d'entraînement
rnn_model.fit(X_train, y_train)

# Prédisez les niveaux de difficulté sur l'ensemble de test
y_pred_rnn = rnn_model.predict(X_test)

# Évaluez la performance du modèle
accuracy_rnn = rnn_model.score(X_test, y_pred_rnn)
print(classification_report(y_test, y_pred_rnn))

del rnn_model

              precision    recall  f1-score   support

           0       0.61      0.61      0.61       166
           1       0.40      0.40      0.40       158
           2       0.36      0.43      0.39       166
           3       0.37      0.36      0.36       153
           4       0.38      0.57      0.46       152
           5       0.50      0.19      0.27       165

    accuracy                           0.43       960
   macro avg       0.44      0.43      0.42       960
weighted avg       0.44      0.43      0.42       960

